<a href="https://colab.research.google.com/github/hsandaver/hsandaver/blob/main/linked_data_explorer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary libraries
!pip install rdflib pyvis ipywidgets

# Import necessary libraries
import rdflib
from google.colab import files
from pyvis.network import Network
import ipywidgets as widgets
from IPython.display import display

# Function to upload files
def upload_files():
    uploaded = files.upload()
    for filename in uploaded.keys():
        print(f"Uploaded file: {filename}")
    return list(uploaded.keys())[0]  # Return the filename for further processing

# Function to load JSON-LD data
def load_jsonld(file_path):
    g = rdflib.Graph()
    g.parse(file_path, format='json-ld')
    return g

# Function to extract relationships from the RDF graph
def extract_relationships(graph):
    relationships = []
    for subj, pred, obj in graph:
        relationships.append((subj, pred, obj))
    return relationships

# Function to filter relationships based on the selected language
def filter_by_language(relationships, selected_language):
    filtered_relationships = []
    for subj, pred, obj in relationships:
        # Filter based on object language tag (assuming labels or literals contain language tags)
        if isinstance(obj, rdflib.Literal) and obj.language == selected_language:
            filtered_relationships.append((subj, pred, obj))
        # Keep non-literal objects (e.g., URIs)
        elif not isinstance(obj, rdflib.Literal):
            filtered_relationships.append((subj, pred, obj))
    return filtered_relationships

# Interactive visualization using pyvis
def visualize_graph_interactive(relationships):
    # Create a pyvis Network object
    net = Network(notebook=True, height="750px", width="100%", bgcolor="#222222", font_color="white")

    # Add nodes and edges from the relationships
    for subj, pred, obj in relationships:
        net.add_node(str(subj), label=str(subj))
        net.add_node(str(obj), label=str(obj))
        net.add_edge(str(subj), str(obj), title=str(pred))

    # Set physics to False to prevent automatic animation movement
    net.set_options("""
    var options = {
      "physics": {
        "enabled": false
      }
    }
    """)

    # Generate and display the network
    net.show("linked_data_network.html")

# Dropdown to select the language
def create_language_dropdown(relationships):
    # Extract all unique languages from the literals in the relationships
    languages = set()
    for _, _, obj in relationships:
        if isinstance(obj, rdflib.Literal) and obj.language:
            languages.add(obj.language)

    # Create a dropdown widget
    language_dropdown = widgets.Dropdown(
        options=list(languages),
        description='Language:',
        disabled=False
    )

    return language_dropdown

# Main function to run the Linked Data tool with language filtering
def run_colab_ld_tool_interactive():
    print("Please upload a JSON-LD file")
    jsonld_file = upload_files()  # Upload and get the filename
    graph = load_jsonld(jsonld_file)  # Load the JSON-LD into an RDF graph
    relationships = extract_relationships(graph)  # Extract relationships

    # Create a language dropdown for the user
    language_dropdown = create_language_dropdown(relationships)
    display(language_dropdown)

    # Define what happens when a language is selected
    def on_language_change(change):
        selected_language = change['new']
        print(f"Filtering relationships for language: {selected_language}")
        filtered_relationships = filter_by_language(relationships, selected_language)
        visualize_graph_interactive(filtered_relationships)

    # Observe changes in the dropdown selection
    language_dropdown.observe(on_language_change, names='value')

# Run the interactive tool with language filtering
run_colab_ld_tool_interactive()
